# DF8 Case 6 Solutions

Muhammad Fadlil Ismail

## Problems

Download the February 2021 data from TLC Trip Record website

(https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) and use

Pyspark to analyze and answer these questions below. Upload your script

into Github or Gdrive.

## Import Library and Spark Session

In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

  Stored in directory: C:\Users\Muhammad Fadlil\AppData\Local\pip\Cache\wheels\e5\cc\9a\0c20ee0940a9e80053edfe2270daee438f36037e1ff041c0ec
Successfully built pyspark


## Import Dataset

In [5]:
df = spark.read.parquet("fhv_tripdata_2021-02.parquet")

Create view from dataset

In [6]:
df.createOrReplaceTempView("ParquetTable")

This is the dataset looks like:

In [11]:
spark.sql("select * from ParquetTable limit 5").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00013|2021-02-01 07:01:00|2021-02-01 08:33:00|        null|        null|   null|                B00014|
|     B00021         |2021-02-01 07:55:40|2021-02-01 08:06:20|       173.0|        82.0|   null|       B00021         |
|     B00021         |2021-02-01 07:14:03|2021-02-01 07:28:37|       173.0|        56.0|   null|       B00021         |
|     B00021         |2021-02-01 07:27:48|2021-02-01 07:35:45|        82.0|       129.0|   null|       B00021         |
|              B00037|2021-02-01 07:12:50|2021-02-01 07:26:38|        null|       225.0|   null|                B00037|
+--------------------+------------------

### Problem 1 : How many taxi trips were there on February 15?

In [20]:
spark.sql("select cast(pickup_datetime as date), COUNT(*) from ParquetTable where extract(day from pickup_datetime) = 15 group by 1").show()

+---------------+--------+
|pickup_datetime|count(1)|
+---------------+--------+
|     2021-02-15|   35709|
+---------------+--------+



### Problem 2 : Find the longest trip for each day ?

In [36]:
spark.sql("select cast(pickup_datetime as date), max((bigint(to_timestamp(dropOff_datetime))-bigint(to_timestamp(pickup_datetime)))/60) as time_delta_minutes from ParquetTable group by 1 order by 1").show()

+---------------+------------------+
|pickup_datetime|time_delta_minutes|
+---------------+------------------+
|     2021-02-01|           46290.0|
|     2021-02-02|1390.7833333333333|
|     2021-02-03|1246.1666666666667|
|     2021-02-04| 40034.88333333333|
|     2021-02-05|          110919.0|
|     2021-02-06| 2752.633333333333|
|     2021-02-07|1306.1166666666666|
|     2021-02-08| 9424.916666666666|
|     2021-02-09|1459.9833333333333|
|     2021-02-10|1407.7166666666667|
|     2021-02-11|3219.8166666666666|
|     2021-02-12|            4344.0|
|     2021-02-13| 8422.683333333332|
|     2021-02-14|            1519.4|
|     2021-02-15|          14670.15|
|     2021-02-16|            4816.1|
|     2021-02-17| 4284.783333333334|
|     2021-02-18|2749.0333333333333|
|     2021-02-19|           9012.15|
|     2021-02-20|2701.4666666666667|
+---------------+------------------+
only showing top 20 rows



### Problem 3 : Find Top 5 Most frequent `dispatching_base_num` ?

In [10]:
spark.sql("select dispatching_base_num, count(*) as number_of_dispatching_base_num from ParquetTable group by 1 order by 2 desc limit 5").show()

+--------------------+------------------------------+
|dispatching_base_num|number_of_dispatching_base_num|
+--------------------+------------------------------+
|              B00856|                         35077|
|              B01312|                         33089|
|              B01145|                         31114|
|              B02794|                         30397|
|              B03016|                         29794|
+--------------------+------------------------------+



### Problem 4 : Find Top 5 Most common location pairs (PUlocationID and DOlocationID)?

In [14]:
spark.sql("select PUlocationID, DOlocationID, count(*) as countPUDUpairs from ParquetTable where PUlocationID is not null and DOlocationID is not null group by 1,2 order by 3 desc limit 5").show()

+------------+------------+--------------+
|PUlocationID|DOlocationID|countPUDUpairs|
+------------+------------+--------------+
|       206.0|       206.0|          2374|
|       221.0|       206.0|          2112|
|       129.0|       129.0|          1902|
|         7.0|         7.0|          1829|
|       179.0|       179.0|          1736|
+------------+------------+--------------+

